# *In silico* evolution of terminators

Install EUGENe

In [1]:
! pip install -q eugene-tools torch==1.11.0 torchtext==0.12.0 torchaudio==0.11.0

## Import modules and define functions

Import the required modules:

In [2]:
import numpy as np
import eugene as eu
import pandas as pd
import glob
import os
import re
import torch
from tqdm.notebook import tqdm

INFO:pytorch_lightning.utilities.seed:Global seed set to 13


GPU is available: True
Number of GPUs: 1
Current GPU: 0
GPUs: Tesla T4


Connect to my Google Drive and go to the correct folder:

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/terminators

Mounted at /content/gdrive
/content/gdrive/MyDrive/terminators


Configure EUGENe directories:

In [4]:
eu.settings.config_dir = "./configs" # Directory to specify when you want to load a model from a config file
eu.settings.dataset_dir = "./dataset" # Directory where EUGENe will download datasets to
eu.settings.logging_dir = "./logs" # Directory where EUGENe will save Tensorboard training logs and model checkpoints to
eu.settings.output_dir = "./output" # Directory where EUGENe will save output files to
eu.settings.figure_dir = "./figures" # Directory to specify to EUGENe to save figures to
eu.settings.seed = 1 # set global seed

INFO:pytorch_lightning.utilities.seed:Global seed set to 1


Define a function to generate all possible point mutations in a one-hot encoded sequence:

In [5]:
bases = eu.pp.ohe_seq('ACGT')

def mutate_seq(sequence):
    mutated_seqs = [sequence]
    for pos in range((sequence.shape[1])):
        for base in bases:
            if not(np.array_equal(sequence[:, pos], base)):
                mut_seq = np.concatenate((sequence[:, :pos], base.reshape(-1, 1), sequence[:, pos+1:]), axis = 1)
                mutated_seqs.append(mut_seq)
                
    mutated_seqs = np.stack(mutated_seqs)
    
    return(mutated_seqs)

Define a function to score sequences with models for both expression systems and pick the best sequence for the indicated system:

In [6]:
def score_seqs(sequences, system):
    with torch.no_grad():
        sequences = torch.Tensor(sequences).to(device)
        predictions = model(sequences)
        predictions = predictions.cpu().detach().numpy()
        
    if system == 'tobacco':
        best_id = predictions[:, 0].argmax()
    if system == 'maize':
        best_id = predictions[:, 1].argmax()
    if system == 'both':
        best_id = np.sum(predictions, axis = 1).argmax()

    best_seq = sequences[best_id].cpu().detach().numpy()
    pred_tobacco = predictions[best_id, 0]
    pred_maize = predictions[best_id, 1]
    
    return(best_seq, pred_tobacco, pred_maize)

Define a function to remove restriction enzyme sites from a sequence by mutating the site and picking the best performing sequence:

In [7]:
# Define restriction enzyme sites as regular expression.
# also considers sites that would be formed with the 5' cloning scar (ends on a G)
REsites = re.compile(r'((G|C|^)GTCTC)|((G|^)AGAC(C|G))')

# helper function to mutate sequences between 'start' and 'stop' 
def mutate_region(sequences, start, stop):
    mut_seqs = []
    
    for sequence in sequences:
        for pos in range(start, stop):
            for base in ['A', 'C', 'G', 'T']:
                if sequence[pos] != base:
                    mut_seq = sequence[:pos] + base + sequence[pos+1:]
                    mut_seqs.append(mut_seq)
                
    return (mut_seqs)

# function to find all matches to restriction enzyme sites, mutate them, and select the best performing sequence
def fix_REsite(sequence, system):
    matches = REsites.finditer(sequence)
    mut_seqs = [sequence]
    
    # introduce mutations to destroy RE sites
    for match in matches:
        mut_seqs = mutate_region(mut_seqs, match.start(), match.end())
        
    # filter out sequences where a new RE site was introduced
    mut_seqs = [seq for seq in mut_seqs if not REsites.search(seq)]
    
    # one-hot encode mutated sequences
    one_hot_seqs = eu.pp.ohe_seqs(mut_seqs, verbose = False)
    
    # score sequences and pick best
    fixed_seq, tobacco, maize = score_seqs(one_hot_seqs, system)
    
    # reverse one-hot encoding of fixed sequence
    fixed_seq = eu.pp.decode_seq(fixed_seq)
        
    return(fixed_seq, tobacco, maize)

## Load models and prepare data

Load the model:

In [8]:
from DenseNet import DenseNet

config_name = 'DenseNet_terminators'
config_file = os.path.join(eu.settings.config_dir, config_name + '.yaml')

model_file = glob.glob(os.path.join(eu.settings.logging_dir, 'ssDenseNet_regression', config_name, 'checkpoints', '*'))[0]
model = DenseNet.load_from_checkpoint(model_file)

Set the model to evaluation mode and send it to the GPU if available:

In [9]:
model.eval()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device_name = 'GPU' if torch.cuda.is_available() else 'CPU'
model.to(device)

print('Model sent to:', device_name)

Model sent to: GPU


Load the data:

In [10]:
data_term = pd.read_csv('terminator_data.tsv', sep = '\t', header = 0).set_index('id')

Randomly select terminators for evolution from both Arabidopsis and maize from the test set: 

In [11]:
n_seqs = 111

AtZm_data = data_term[(data_term['set'] == 'test') & (data_term.species.isin(['Arabidopsis', 'Maize']))]

start_terminators = AtZm_data.groupby('species').sample(n = n_seqs, random_state = eu.settings.seed)

One-hot encode start sequences:

In [12]:
start_seqs = eu.pp.ohe_seqs(start_terminators['sequence'])

One-hot encoding sequences:   0%|          | 0/222 [00:00<?, ?it/s]

Predict promoter strength of start sequences:

In [13]:
with torch.no_grad():
    start_tensor = torch.Tensor(start_seqs).to(device)
    predictions = model(start_tensor)
    predictions = predictions.cpu().detach().numpy()

Set up dataframe for results:

In [14]:
evolution_data = pd.DataFrame(data = {
    'round' : 0,
    'species' : start_terminators.species,
    'origin' : start_terminators.index,
    'opt_for' : 'start',
    'sequence' : list(start_seqs),
    'prediction_tobacco' : predictions[:, 0],
    'prediction_maize' : predictions[:, 0]
})

## Perform the evolution

Perform iterative evolution of the promoter sequences:

In [15]:
n_rounds = 10

for system in ['tobacco', 'maize', 'both']:
    for thisround in tqdm(range(1, n_rounds + 1), desc = 'Optimizing terminators for ' + system):        
        lastround = thisround - 1
        data = evolution_data[evolution_data['round'] == lastround]

        if thisround > 1:
            data = data[data['opt_for'] == system]
            
        if data.empty:
            break
            
        species = data['species']
        origins = data['origin']
        sequences = data['sequence']
            
        for sp, origin, sequence in zip(species, origins, sequences):
            mut_seqs = mutate_seq(sequence)
            opt_seq, tobacco, maize = score_seqs(mut_seqs, system)
            
            if not(np.array_equal(opt_seq, sequence)):
                evolution_data.loc[len(evolution_data)] = [thisround, sp, origin, system, opt_seq, tobacco, maize]

Optimizing terminators for tobacco:   0%|          | 0/10 [00:00<?, ?it/s]

Optimizing terminators for maize:   0%|          | 0/10 [00:00<?, ?it/s]

Optimizing terminators for both:   0%|          | 0/10 [00:00<?, ?it/s]

Reverse one-hot encoding:

In [16]:
evolution_data['sequence'] = eu.pp.decode_seqs(evolution_data['sequence'].tolist())

Decoding sequences:   0%|          | 0/6882 [00:00<?, ?it/s]

Annotate if the sequence contains a restriction enzyme site:

In [17]:
evolution_data['REsite'] = [bool(REsites.search(x)) for x in evolution_data['sequence']]

Select sequences that need to be fixed (third or final round and contains a restriction enzyme site):

In [18]:
fix_seqs = evolution_data[((evolution_data['round'] == 3) | (evolution_data['round'] == max(evolution_data['round']))) & (evolution_data['REsite'])].copy()

Fix sequences:

In [19]:
if len(fix_seqs) > 0:
    fix_seqs[['sequence', 'prediction_tobacco', 'prediction_maize']] = [fix_REsite(seq, sys) for (seq, sys) in zip(fix_seqs['sequence'], fix_seqs['opt_for'])]
    evolution_data = fix_seqs.combine_first(evolution_data)

Convert 'round' column to integer:

In [20]:
evolution_data = evolution_data.astype({'round' : int})

Save data to a file

In [21]:
evolution_data.to_csv('evolution_data.tsv', sep = '\t', index = False)